# **Music Listening During the Coronavirus: Personality Differences**

In [1]:
import numpy as np, pandas as pd, os, sys, itertools, re, warnings, math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings(action='ignore')
os.chdir("/Users/home/Desktop/Research/Coronavirus/Public GitHub")
%run Coronavirus_Music_Functions.ipynb

In [2]:
with open("/Users/home/Desktop/Research/Coronavirus/Public GitHub/factors_all_data.csv", encoding="utf8", errors='ignore') as file:
    all_data = pd.read_csv(file)
        
all_data.rename(columns={'Demographics_General_Gender': 'Gender'}, inplace=True)
all_data.shape

(5113, 238)

In [3]:
# reorder columns
id_col = all_data.pop('ID')
country_col = all_data.pop('Country')
gender_col = all_data.pop('Gender')

all_data.insert(0, 'Gender', gender_col)
all_data.insert(0, 'Country', country_col)
all_data.insert(0, 'ID', id_col)

In [4]:
# only keep people in listening and both groups
data = all_data[all_data['Survey Info_Group'].isin(['both','listening only'])].reset_index(drop=True)
data['Survey Info_Group'].value_counts()

both              3003
listening only    1669
Name: Survey Info_Group, dtype: int64

In [5]:
# get separate dataframes for country and factor scores
country = pd.get_dummies(data['Country'], prefix='Country')
factors = data[['F1','F2','F3','F4','F5']]

In [6]:
# only keep relevant columns
drop_cols = ['F1','F2','F3','F4','F5', 'Making Music', 'Formats', 'Demographics','Survey Info_Group','Country']
col_list = [string for string in data.columns if not any(substring in string for substring in drop_cols)]
data = data[col_list]

data.columns = data.columns.str.replace("Music Listening_", "")

In [7]:
# final df
df = pd.concat([factors, country, data], axis = 1, sort = False).reset_index(drop = True)
# df.drop(['Music Engagement_Engaging With Music Differently YN'], axis=1, inplace = True)

df.shape

(4672, 103)

#### **Factor DFs**

In [8]:
# set cutoff scores for defining 'high' and 'low' scorers
high_cutoff = 75
low_cutoff = 25

f1 = split_into_high_low(df, 'F1', low_cutoff, high_cutoff)
f2 = split_into_high_low(df, 'F2', low_cutoff, high_cutoff)
f3 = split_into_high_low(df, 'F3', low_cutoff, high_cutoff)
f4 = split_into_high_low(df, 'F4', low_cutoff, high_cutoff)
f5 = split_into_high_low(df, 'F5', low_cutoff, high_cutoff)

#### **Factor 1: Negative Emotions**

In [9]:
factor1_summary = factor_summary(f1)
cols = [i for i in factor1_summary.index if not i.startswith('Activities')]
factor1_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.67,0.02,5.25,1.48,4.57,1.45
Situations_Dining,0.50,0.24,4.39,1.46,3.89,1.22
Situations_Doing Nothing Else,0.66,0.07,5.07,1.45,4.40,1.39
Situations_Evening,0.61,0.10,4.70,1.45,4.09,1.35
Situations_Exercising,0.62,0.05,4.96,1.43,4.34,1.38
Situations_Housework,0.65,0.03,5.10,1.45,4.45,1.41
Situations_Morning,0.50,0.17,4.56,1.51,4.06,1.34
Situations_Other Activities,0.73,0.05,5.05,1.42,4.31,1.37
Situations_Sexual Activity,0.45,0.26,4.34,1.37,3.89,1.11
Situations_Traveling,0.52,0.17,4.61,1.67,4.08,1.50


#### **Factor 2: Living Situation**

In [10]:
factor2_summary = factor_summary(f2)
cols = [i for i in factor2_summary.index if not i.startswith('Activities')]
factor2_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.76,0.33,5.12,1.50,4.35,1.17
Situations_Dining,0.13,0.44,4.04,1.39,3.91,0.95
Situations_Doing Nothing Else,0.65,0.45,4.89,1.52,4.25,1.07
Situations_Evening,0.57,0.60,4.51,1.54,3.94,0.93
Situations_Exercising,0.93,0.46,4.95,1.48,4.02,1.02
Situations_Housework,0.58,0.41,4.87,1.51,4.29,1.10
Situations_Morning,0.19,0.53,4.22,1.52,4.03,0.99
Situations_Other Activities,0.63,0.40,4.83,1.45,4.20,1.05
Situations_Sexual Activity,0.14,0.43,4.06,1.28,3.91,0.85
Situations_Traveling,0.28,0.74,4.29,1.76,4.01,1.02


#### **Factor 3: Positive Emotions**

In [11]:
factor3_summary = factor_summary(f3)
cols = [i for i in factor3_summary.index if not i.startswith('Activities')]
factor3_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.33,0.05,5.05,1.48,4.72,1.43
Situations_Dining,0.76,0.33,4.54,1.47,3.79,1.13
Situations_Doing Nothing Else,0.36,0.12,4.88,1.48,4.52,1.36
Situations_Evening,0.69,0.26,4.77,1.48,4.08,1.22
Situations_Exercising,0.49,0.18,4.90,1.48,4.40,1.30
Situations_Housework,0.43,0.12,4.95,1.48,4.53,1.36
Situations_Morning,0.69,0.28,4.67,1.52,3.98,1.25
Situations_Other Activities,0.41,0.15,4.90,1.47,4.48,1.32
Situations_Sexual Activity,0.62,0.49,4.46,1.46,3.85,0.97
Situations_Traveling,0.66,0.14,4.68,1.60,4.02,1.45


#### **Factor 4: Age**

In [12]:
factor4_summary = factor_summary(f4)
cols = [i for i in factor4_summary.index if not i.startswith('Activities')]
factor4_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.23,0.01,4.96,1.41,4.73,1.40
Situations_Dining,0.20,0.13,4.22,1.32,4.02,1.20
Situations_Doing Nothing Else,0.30,0.00,4.83,1.39,4.53,1.38
Situations_Evening,0.21,0.10,4.46,1.34,4.25,1.25
Situations_Exercising,0.35,0.08,4.79,1.38,4.44,1.30
Situations_Housework,0.38,0.09,4.88,1.41,4.50,1.32
Situations_Morning,0.31,0.13,4.40,1.38,4.10,1.25
Situations_Other Activities,0.32,0.10,4.79,1.37,4.47,1.27
Situations_Sexual Activity,0.26,0.20,4.19,1.23,3.93,1.03
Situations_Traveling,0.15,0.25,4.30,1.65,4.15,1.41


#### **Factor 5: City Type**

In [13]:
factor5_summary = factor_summary(f5)
cols = [i for i in factor5_summary.index if not i.startswith('Activities')]
factor5_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,-0.26,-0.18,4.63,1.29,4.89,1.47
Situations_Dining,-0.10,-0.25,3.97,1.11,4.06,1.37
Situations_Doing Nothing Else,-0.17,-0.24,4.50,1.25,4.67,1.49
Situations_Evening,-0.15,-0.22,4.17,1.19,4.32,1.40
Situations_Exercising,-0.26,-0.15,4.35,1.25,4.62,1.40
Situations_Housework,-0.19,-0.19,4.51,1.29,4.70,1.48
Situations_Morning,-0.18,-0.20,4.07,1.22,4.25,1.42
Situations_Other Activities,-0.25,-0.16,4.44,1.26,4.68,1.42
Situations_Sexual Activity,-0.12,-0.26,3.95,1.00,4.07,1.26
Situations_Traveling,-0.26,-0.19,4.03,1.39,4.29,1.58
